# PaLM2 Few-Shot Prompting and RAG Proof-of-Concept

## Objective

##### This notebook will serve as a local test ground to evaluate the PaLM2 fine-tuned model with RAG for column-mapping

## Libraries

In [1]:
# General
import csv
import numpy as np
from typing import Dict, List, Optional
import lark

# langchain dependencies
from langchain import hub

# from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain.document_loaders.base import BaseLoader
from langchain.docstore.document import Document
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.vectorstores import Chroma

# Vectorstore and Embeddings
from langchain_google_vertexai import VertexAI
from langchain_google_vertexai import VertexAIEmbeddings

## Configuring Environment Settings

In [2]:
# Project definitions
# PROJECT_ID = "dlabs-intelligent-col-mapper"  # @param {type:"string"}
# REGION = "asia-southeast1"  # @param {type: "string"}

! gcloud auth application-default login

## RAG Integration

### Customized CSVLoader

In [2]:
class CSVLoader(BaseLoader):
    """Loads a CSV file into a list of documents.

    Each document represents one row of the CSV file. Every row is converted into a
    key/value pair and outputted to a new line in the document's page_content.

    The source for each document loaded from csv is set to the value of the
    `file_path` argument for all doucments by default.
    You can override this by setting the `source_column` argument to the
    name of a column in the CSV file.
    The source of each document will then be set to the value of the column
    with the name specified in `source_column`.

    Output Example:
        .. code-block:: txt

            column1: value1
            column2: value2
            column3: value3
    """

    def __init__(
        self,
        file_path: str,
        source_column: Optional[str] = None,
        metadata_columns: Optional[List[str]] = None,   # < ADDED
        csv_args: Optional[Dict] = None,
        encoding: Optional[str] = None,
    ):
        self.file_path = file_path
        self.source_column = source_column
        self.encoding = encoding
        self.csv_args = csv_args or {}
        self.metadata_columns = metadata_columns        # < ADDED

    def load(self) -> List[Document]:
        """Load data into document objects."""

        docs = []
        with open(self.file_path, newline="", encoding=self.encoding) as csvfile:
            csv_reader = csv.DictReader(csvfile, **self.csv_args)  # type: ignore
            for i, row in enumerate(csv_reader):
                content = "\n".join(f"{k.strip()}: {v.strip()}" for k, v in row.items())
                try:
                    source = (
                        row[self.source_column]
                        if self.source_column is not None
                        else self.file_path
                    )
                except KeyError:
                    raise ValueError(
                        f"Source column '{self.source_column}' not found in CSV file."
                    )
                metadata = {"source": source, "row": i}
                # ADDED TO SAVE METADATA
                if self.metadata_columns:
                    # print(self.metadata_columns)
                    for k, v in row.items():
                        #print(k)
                        if k in self.metadata_columns:
                            #print(k)
                            metadata[k] = v
                            #print(metadata[k])
                # END OF ADDED CODE
                doc = Document(page_content=content, metadata=metadata)
                docs.append(doc)

        return docs

### Loading Data

In [3]:
excel_file = r"C:\Users\gil.cruzada\Desktop\Column-Mapper\dev-ground\cm-dev-dataset\copy_dm.csv"
loader = CSVLoader(file_path = excel_file, source_column = 'target_column', metadata_columns = ['data_type','target_table'])
data = loader.load()
# data

In [4]:
# Defining embedding object
embeddings = VertexAIEmbeddings(model_name='textembedding-gecko@001')

## Vector Database through Chroma

In [5]:
# Vectore store
chroma_vectorstore = Chroma.from_documents(data, embeddings)

In [6]:
# Create Retriever
chroma_retreiver = chroma_vectorstore.as_retriever()

### Model loading

In [41]:
llm=VertexAI(model_name="text-bison@001",temperature=0)
metadata_field_info=[
     AttributeInfo(
        name="data_type",
        description="datatype of the target_column", 
        type="string", 
    ),
    AttributeInfo(
        name="target_table",
        description="designated table where the target_column is located", 
        type="string", 
    ),
]
document_content_description = "The data model where source_columns are being mapped to target_columns based on data_type, description, and target_column"
retriever = SelfQueryRetriever.from_llm(
    llm, chroma_vectorstore, document_content_description, metadata_field_info, search_kwargs={"k": 20},verbose=True
)

In [12]:
rag_prompt = hub.pull("rlm/rag-prompt")

### with RAG prompt

In [38]:
prompt = """
Given the input below you are tasked with mapping a source column to its target column. You must take into account 
the sample values, data type, source_table, and source column in your task of column mapping. 
Take priority in mapping a source column first to its target column. The structure of your output should look like the example outputs below. 
Strictly maintain the output structure

input: source_column: branch, source_table: journal_header, data_type: integer, sample: ["1927","1563", "410"]
output: target_column: branch_key, target_table: fct_header
      
input: source_column: company, source_table: dim_store, data_type: string, sample: ["MARKET STRATEGIC FIRM, INC.", "MERIDIEN BUSINESS LEADER, INC.", "MERCANTILE STORES GROUP, INC."]
output: target_column: company_name, target_table: dim_company
      
input: source_column: line_amt1, source_table: journal_details, data_type: float, sample: ["185.25", "122.50", "95.18"]
output: target_column: amount, target_table: fct_sku
      
input: source_column: upload_date, source_table: csa_fct_header, data_type: timestamp, sample: ["2023-04-29 03:10:46.110000", "2023-08-26 03:02:31.057000", "2023-09-18 03:50:44.760000"]
output: target_column: created_date, target_table: fct_transaction

input: source_column: arc_date, source_table: dim_store, data_type: date, sample: ["2023-06-22", "2023-12-28", "2021-10-38"]
output: target_column: created_date, target_table: fct_transaction
      
input: source_column: qty, source_table: csa_fct_merchandise, data_type: float, sample: ["27.0", "1.0", "13.0"]
output: 
"""

In [73]:
prompt_2 = """
Given the input below you are tasked with mapping a source column to its target column. You must take into account 
the sample values, data type, source_table, and source column in your task of column mapping. 
The structure of your output should look like the example outputs below. Strictly maintain the output structure

input: source_column: company, source_table: dim_store, data_type: string, sample: ["MARKET STRATEGIC FIRM, INC.", "MERIDIEN BUSINESS LEADER, INC.", "MERCANTILE STORES GROUP, INC."]
output:
"""

In [74]:
# RAG chain 
from langchain.schema.runnable import RunnablePassthrough
rag_chain = (
    {"context": chroma_vectorstore.as_retriever(), "question": RunnablePassthrough()} 
    | rag_prompt 
    | llm 
)

rci_output = rag_chain.invoke(prompt_2)
print("rci_output is: ", rci_output)

rci_output is:  The column company in the source table dim_store is mapped to the column company_name in the target table dim_company.
